In [1]:
# !wget https://huggingface.co/datasets/mesolitica/wikipedia/resolve/main/wikipedia-2023-06-10.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-my-website/resolve/main/dewanbahasa.jendeladbp.my.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-malaysian-hansard/resolve/main/hansard.jsonl
# !wget https://huggingface.co/datasets/mesolitica/crawl-lom-agc-gov-my/resolve/main/dedup.jsonl -O agc.jsonl
# !pip3.8 install git+https://github.com/huggingface/transformers.git
# !pip3.8 install accelerate==0.21.0
# !pip3.8 install tokenizers==0.13.3

In [2]:
import malaya

language_detection = malaya.language_detection.fasttext(model = 'mesolitica/fasttext-language-detection-ms-id')

2023-07-26 21:32:56.444334: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-26 21:32:56.516115: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-07-26 21:32:57.045528: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-07-26 21:32:57.045560: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not l


===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching /usr/local/cuda/lib64...
ERROR: /usr/bin/python3: undefined symbol: cudaRuntimeGetVersion
CUDA SETUP: libcudart.so path is None
CUDA SETUP: Is seems that your cuda installation is not in your path. See https://github.com/TimDettmers/bitsandbytes/issues/85 for more information.
CUDA SETUP: CUDA version lower than 11 are currently not supported for LLM.int8(). You will be only to use 8-bit optimizers and quantization routines!!
CUDA SETUP: Highest compute capability among GPUs detected: 8.6
CUDA SETUP: Detected CUDA version 00
CUDA SETUP: Loading binary /home/husein/.local/lib/python3.8/site-packages/bitsandbytes/libbitsandbytes_cpu.so...


/home/husein/.local/lib/python3.8/site-packages/bitsandbytes/cuda_setup/main.py:136: UserWarning: WARNING: The following directories listed in your path were found to be non-existent: {PosixPath('mr=\\E'), PosixPath('is=\\E)0'), PosixPath('k8=\\E[19~'), PosixPath('RI=\\E[%dC'), PosixPath('k0=\\E[10~'), PosixPath('us=\\E'), PosixPath('li#26'), PosixPath('st=\\EH'), PosixPath('km'), PosixPath('ae=\\E(B'), PosixPath('kB=\\E[Z'), PosixPath('cr=^M'), PosixPath('mb=\\E'), PosixPath('F2=\\E[24~'), PosixPath('ho=\\E[H'), PosixPath('kP=\\E[5~'), PosixPath('kl=\\EOD'), PosixPath('it#8'), PosixPath('kd=\\EOB'), PosixPath('so=\\E'), PosixPath('k3=\\EOR'), PosixPath('kI=\\E[2~'), PosixPath('am'), PosixPath('LE=\\E[%dD'), PosixPath('AB=\\E[4%dm'), PosixPath('k6=\\E[17~'), PosixPath('cs=\\E[%i%d;%dr'), PosixPath('im=\\E[4h'), PosixPath('kH=\\E[4~'), PosixPath('pt'), PosixPath('ta=^I'), PosixPath('ct=\\E[3g'), PosixPath('do=^J'), PosixPath('G0'), PosixPath('dc=\\E[P'), PosixPath('se=\\E'), PosixPath('

In [3]:
from transformers import AutoTokenizer
import json

partition_size = 512
tokenizer = AutoTokenizer.from_pretrained('meta-llama/Llama-2-7b-hf')

In [4]:
import re

rejected = [
    'Internal Server Error',
    '404',
    '__NOEDITSECTION__',
    'enter your username and password',
    'Login',
    'forgotten your password',
    'cookies enabled'
]

def replace_multiple(input_string, pattern =r"\s{3,}", replace = '   '):
    return re.sub(pattern, replace, input_string)

def replace(string):
    string = replace_multiple(string.replace('…', '.'))
    string = replace_multiple(string, pattern = r"\.{3,}", replace = '...')
    return string

def partition(tokens, partition_size=partition_size, check_language = True):
    tokens = [tokens[i: i + partition_size] for i in range(0, len(tokens), partition_size)]
    selected = []
    for t in tokens:
        t = tokenizer.decode(t)
        if any([r in t for r in rejected]):
            continue
        if check_language and language_detection.predict([t])[0] != 'standard-malay':
            continue
        selected.append(t)
        
    return selected

In [5]:
fopen_l = open('partition-512.jsonl', 'w')

In [6]:
from tqdm import tqdm

In [7]:
with open('agc.jsonl') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)
        data = replace(data)
        if len(data.split()) > 50000:
            continue
        tokenized = tokenizer.encode(data)
        partitioned = partition(tokenized)
        for p in partitioned:
            o = {
                'prompt_input': None,
                'input': p,
                'output': None
            }
            fopen_l.write(f'{json.dumps(o)}\n')

1739it [00:46, 37.19it/s] 


In [8]:
with open('/home/husein/ssd3/eprints/filtered.jsonl') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)
        data = replace(data)
        if len(data.split()) > 15000:
            continue
        tokenized = tokenizer.encode(data)
        partitioned = partition(tokenized)
        for p in partitioned:
            o = {
                'prompt_input': None,
                'input': p,
                'output': None
            }
            fopen_l.write(f'{json.dumps(o)}\n')

40612it [20:25, 33.15it/s]


In [9]:
with open('dewanbahasa.jendeladbp.my.jsonl') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)
        if 'p' not in data:
            continue
        if data['p'] is None:
            continue
        data = '\n'.join(data['p'][2:-1])
        data = replace(data)
        tokenized = tokenizer.encode(data)
        partitioned = partition(tokenized)
        for p in partitioned:
            o = {
                'prompt_input': None,
                'input': p,
                'output': None
            }
            fopen_l.write(f'{json.dumps(o)}\n')

4580it [00:11, 388.52it/s]


In [10]:
with open('wikipedia-2023-06-10.jsonl') as fopen:
    for l in tqdm(fopen):
        data = json.loads(l)['text']
        data = replace(data)
        tokenized = tokenizer.encode(data)
        partitioned = partition(tokenized)
        for p in partitioned:
            o = {
                'prompt_input': None,
                'input': p,
                'output': None
            }
            fopen_l.write(f'{json.dumps(o)}\n')

434321it [04:38, 1561.36it/s]


In [11]:
fopen_l.close()

In [12]:
# with open('hansard.jsonl') as fopen:
#     for l in tqdm(fopen):
#         data = json.loads(l)['original']
#         data = replace(data)
#         tokenized = tokenizer.encode(data)
#         partitioned = partition(tokenized)
#         for p in partitioned:
#             o = {
#                 'prompt_input': None,
#                 'input': p,
#                 'output': None
#             }
#             fopen_l.write(f'{json.dumps(o)\n}')